# TP3: Análisis de datos RRBS

**Nota**: Este cuaderno asume que el [TP0](https://colab.research.google.com/github/VeronicaNoe/EpiEvo/blob/main/cuadernos/TP0_espacio_de_trabajo.ipynb) ha sido completado con éxito.


# Introducción

Analizaremos la metilación del ADN en `clones` de un genotipo de *Vitis vinifera* cv.  Malbec cultivados en dos viñedos contrastantes: Agrelo **(Agr)** y Gualtallary **(Gua)**.

A pesar de que son clones en cada finca, genéticamente idénticas, se observan diferencias fenotípicas entre ellas asociadas a cada localidad.

La réplicas biológicas se denominan:
- **Agr1**, **Agr4**, **Agr7**;
- **Gua3**, **Gua7**, **Gua9**.

<p align="left">
<img src="https://raw.githubusercontent.com/VeronicaNoe/EpiEvo/main/TP_03_AA.
png" width="800"/> 
</p>




# Contenido

**Objetivo:** Familiarizarse con la identificación de citosinas metiladas usando datos RRBS de vid [_Vitis vinifera_](https://www.researchgate.net/profile/Anabella-Varela).


<p align="left">
<img src="https://raw.githubusercontent.com/VeronicaNoe/EpiEvo/main/img/3/TP_3_A.JPG" width="500"/> 
</p>

0.   [Preparación de cuaderno](#step-0)
1.   [Preparación del espacio de trabajo en R](#step-1)
2.   [Análisis de datos obtenidos por *RRBS*](#step-2)
  1.   [Unir archivos](#step-2.1)
  2.   [Explorar datos](#step-2.2)
  3.   [Control de calidad](#step-2.3)
  4.   [Caracterización global de la metilación](#step-2.4)
  5.   [Citosinas diferencialmente metiladas](#step-2.5)
3.   [OPCIONAL: Guardar en Drive](#step-3)



<a name="step-0"></a>
# Preparación del cuaderno

Para trabajar de forma ordenada, se crearán 3 carpetas en colab:
- *rawData*: en donde estarán los archivos necesarios para trabajar (**input**)
- *results*: en donde se guardarán los archivos generados (**output**)
- *plots*: en donde se guardarán los pdf de las figuras (**output**)

In [ ]:
#@title Desde el navegador de la derecha, revisar los directorios
%%bash
mkdir {rawData,plots,results}
rm -r sample_data/ # eliminarmos la carpeta que está por default en colab
ls

drive
plots
rawData
results


En el directorio *rawData* cargaremos los inputs



In [ ]:
#@title Cargar archivos
#@title Descargar archivos en el directorio
%%bash
cp -r /content/drive/MyDrive/EpiEvo/epievo_data/3/* /content/rawData
ls rawData

Agrelo_10A1_trimmed_bismark_bt2.bismark_covered_only.CHG_report.txt
Agrelo_10A1_trimmed_bismark_bt2.bismark_covered_only.CHH_report.txt
Agrelo_10A1_trimmed_bismark_bt2.bismark_covered_only.CpG_report.txt
Agrelo_10A4_trimmed_bismark_bt2.bismark_covered_only.CHG_report.txt
Agrelo_10A4_trimmed_bismark_bt2.bismark_covered_only.CHH_report.txt
Agrelo_10A4_trimmed_bismark_bt2.bismark_covered_only.CpG_report.txt
Agrelo_10A7_trimmed_bismark_bt2.bismark_covered_only.CHG_report.txt
Agrelo_10A7_trimmed_bismark_bt2.bismark_covered_only.CHH_report.txt
Agrelo_10A7_trimmed_bismark_bt2.bismark_covered_only.CpG_report.txt
commonFunctions.R
Gualtallary_10G3_trimmed_bismark_bt2.bismark_covered_only.CHG_report.txt
Gualtallary_10G3_trimmed_bismark_bt2.bismark_covered_only.CHH_report.txt
Gualtallary_10G3_trimmed_bismark_bt2.bismark_covered_only.CpG_report.txt
Gualtallary_10G7_trimmed_bismark_bt2.bismark_covered_only.CHG_report.txt
Gualtallary_10G7_trimmed_bismark_bt2.bismark_covered_only.CHH_report.txt
Gualt

<a name="step-1"></a>
# Preparación del espacio de trabajo en R




In [ ]:
#@title Cargar R
%load_ext rpy2.ipython

In [ ]:
#@title Instalar paquetes, cargar librerías y establecer directorio de trabajo
%%R
install.packages(c("plyr","reshape2","qqman"))
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
#BiocManager::install(c("methylKit","GenomicRanges","IRanges", "genomation"))
BiocManager::install(c("methylKit"))

# Cargar librerias
suppressPackageStartupMessages({
  library("plyr") 
  library("reshape2") 
  library("qqman")
  library("methylKit")
  #library("GenomicRanges")
  #library("IRanges")
  source(file.path("/content/rawData", "commonFunctions.R"))
})

# Definir directorio de trabajo
setwd("/content/results")
getwd()

R[write to console]: Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: also installing the dependency ‘calibrate’


R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/calibrate_1.7.7.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 306782 bytes (299 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[writ

Update all/some/none? [a/s/n]: a


R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/cpp11_0.4.0.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 317563 bytes (310 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]:

[1] "/content/results"


<a name="step-2"></a>
# Análisis de datos obtenidos por *RRBS*

<a name="step-2.1"></a>
## Unir todos los archivos en una sola tabla

Como vimos en el TP anterior (LINK al CUADERNO), para cada muestra tendremos un archivo. 

Para cada contexto, uniremos los archivos.

Hay que definir el nombre de las muestras, los grupos que queremos comparar y el nombre del genoma de referencia utilizado.


In [ ]:
#@title 
%%R
inFile<-list.files(path="/content/rawData",pattern = "report.txt", full.names = TRUE)
sampleNames<-list("Agr1","Agr4","Agr7","Gua3","Gua7","Gua9") #Definir el nombre de las muestras
contr<-c(0,0,0,1,1,1) #Definir los contrastas del diseño exp
refGen<-"Vv12X_44" #Nombre del genoma de Referencia

CG<-list()
CHG<-list()
CHH<-list()
inFileCG<-grep("CpG", inFile, value=TRUE)
inFileCHG<-grep("CHG", inFile, value=TRUE)
inFileCHH<-grep("CHH", inFile, value=TRUE)
for (i in 1:length(inFileCG)){
  CG[[i]] <- inFileCG[i]
  CHG[[i]] <- inFileCHG[i]
  CHH[[i]] <- inFileCHH[i]
}  
CG.cov<-methRead(CG, context = "CpG", sample.id =sampleNames, 
                        treatment =contr, assembly= refGen , pipeline = "bismarkCytosineReport", 
                        mincov = 10)
CHG.cov<-methRead(CHG, context = "CHG", sample.id =sampleNames, 
                   treatment =contr, assembly= refGen , pipeline = "bismarkCytosineReport", 
                   mincov = 10)
CHH.cov<-methRead(CHH, context = "CHH", sample.id =sampleNames, 
                   treatment =contr, assembly= refGen , pipeline = "bismarkCytosineReport", 
                   mincov = 10)


<a name="step-2.2"></a>
## Explorar los datos

Methylkit utiliza listas. Para cada contexto, tendremos una lista con n elementos equivalentes al número de muestras.
Para cada muestra, tendremos una tabla con 7 columnas:
- **chr**: fragmento o cromosoma (del inglés: chromosome) de las muestras
- **start**: posición inicial dentro del fragmento 5'->3'
- **end**: posición final dentro del fragmento 5'->3'
- **strand**: indica si la citosina se encuentra en la hebra líder *(+)* o en la complementaria *(-)*.
- **coverage**: número total de veces que se secuenció esa posición
- **numCs**: indica el número de citosinas metiladas
- **numTs**: indica el número de citosinas no metiladas

In [ ]:
#@title Datos crudos.
%%R
cat("\n\n","\t\t\t\t Vista previa de la tabla:", "\n")
head(CG.cov)[[1]]

<a name="step-2.3"></a>
## Control de calidad


Se filtrarán citosinas con baja y con alta cobertura. Es decir, citosinas que fueron secuenciadas pocas o muchas veces. 

La siguiente figura representa un alineamiento entre el genoma de referencia (rojo) y las lecturas o reads de las muestas (verde). Con la secuenciación y el alineamiento de fragmentos de ADN al genoma de referencia es posible determinar el estado de metilación de las C's.
<p align="left">
<img src="https://raw.githubusercontent.com/VeronicaNoe/EpiEvo/main/TP_03_A.png" width="400"/> 
</p>

La tabla de datos que trabajaremos contiene el conteo de cuántas de C's presentes en el genoma de referencia están efectivamente en los reads. 
Dado que el tratamiento con bisulfito deja intactas las citosinas metiladas, podemos calcular la proporción de citosinas metiladas para cada posición del genoma.

En la siguiente figura, las `C verdes` (reads) alineadas con C rojas (genoma de referencia), son `C metiladas`. Mientras que las `T verdes` alineadas con C rojas, son `C no metiladas` que fueron modificadas por el tratamiento con bisulfito. 
<p align="left">
<img src="https://raw.githubusercontent.com/VeronicaNoe/EpiEvo/main/TP_03_B.png" width="400"/> 
</p>
Si tenemos en cuenta el total de Citosinas secuenciadas (metiladas y no metiladas) podemos determinar la cobertura de esa base y eliminar las que están sub- o sobre-representadas:

- **Baja cobertura**: El mínimo es `10X`, se puede modificar y ser menos estrictos pero esto compromete la confianza en los datos.

- **Alta cobertura**: Los valores de cobertura muy altos también son problemáticos ya que se puede detectar citosinas/posiciones diferencialmente metiladas (DMC/DMC) a aquellas C's en que la diferencia se deba a cobertura.
El umbral depende de los investigadores. 

<p align="left">
<img src="https://raw.githubusercontent.com/VeronicaNoe/EpiEvo/main/TP_03_C.png" width="400"/> 
</p>


`Momento Kahoot!`

<img src="https://raw.githubusercontent.com/VeronicaNoe/EpiEvo/main/Kahoot_img.png" width="100"/> 

https://kahoot.it/challenge/03587781?challenge-id=ada51b00-8ab2-4aa7-8308-209bebbf3c4f_1633430918049

In [ ]:
#@title Distribución de la metilación porcentual
#The following command plots the histogram for percent methylation distribution.
#The figure below is the histogram and numbers on bars denote what percentage of 
#locations are contained in that bin. 
%%R
allContexts <- c("CpG", "CHG", "CHH")
numPlotRows <- 2 #
numPlotCols <- length(allContexts)

for (c in allContexts){
  layout(matrix(1:(numPlotRows*numPlotCols), nrow = numPlotRows, byrow = TRUE))
  for (i in 1:length(sampleNames)){
    if(c=="CpG"){
      getMethylationStats(CG.cov[[i]], plot = T, both.strands = F,labels = F) 
    }else if (c=="CHG"){
      getMethylationStats(CHG.cov[[i]], plot = T, both.strands = F, labels = F)
    }else{
      getMethylationStats(CHH.cov[[i]], plot = T, both.strands = F,labels = F) 
    }
  }
}


for (c in allContexts){
  pdf(paste0("/content/plots/",c,"_metilacionPorcentaje_histograma.pdf"))
  layout(matrix(1:(numPlotRows*numPlotCols), nrow = numPlotRows, byrow = TRUE))
  for (i in 1:length(sampleNames)){
    if(c=="CpG"){
      getMethylationStats(CG.cov[[i]], plot = T, both.strands = F,labels = F) 
    }else if (c=="CHG"){
      getMethylationStats(CHG.cov[[i]], plot = T, both.strands = F, labels = F)
    }else{
      getMethylationStats(CHH.cov[[i]], plot = T, both.strands = F,labels = F) 
    }
  }
  dev.off()
}


In [ ]:
#@title Distribución de los conteos
#We can also plot the read coverage per base information in a similar way,
#again numbers on bars denote what percentage of locations are contained in that bin. 
#Experiments that are highly suffering from PCR duplication bias will have a secondary
#peak towards the right hand side of the histogram.
%%R
allContexts <- c("CpG", "CHG", "CHH")
numPlotRows <- 2 #
numPlotCols <- length(allContexts)

for (c in allContexts){
  layout(matrix(1:(numPlotRows*numPlotCols), nrow = numPlotRows, byrow = TRUE))
  for (i in 1:length(sampleNames)){
    if(c=="CpG"){
      getCoverageStats(CG.cov[[i]], plot = T, both.strands = F,labels = F) 
    }else if (c=="CHG"){
      getCoverageStats(CHG.cov[[i]], plot = T, both.strands = F, labels = F)
    }else{
      getCoverageStats(CHH.cov[[i]], plot = T, both.strands = F,labels = F) 
    }
  }
}

for (c in allContexts){
  pdf(paste0("/content/plots/",c,"_cobertura_histograma.pdf"))
  layout(matrix(1:(numPlotRows*numPlotCols), nrow = numPlotRows, byrow = TRUE))
  for (i in 1:length(sampleNames)){
    if(c=="CpG"){
      getCoverageStats(CG.cov[[i]], plot = T, both.strands = F,labels = F) 
    }else if (c=="CHG"){
      getCoverageStats(CHG.cov[[i]], plot = T, both.strands = F, labels = F)
    }else{
      getCoverageStats(CHH.cov[[i]], plot = T, both.strands = F,labels = F) 
    }
  }
  dev.off()
}


**Filtrado**: 
 - Seleccionamos aquellas citosinas secuenciadas más de 10 veces y menos del max que están *presentes en el **80%** de las muestras*.



In [ ]:
#@title Datos filtrados.
# It might be useful to filter samples based on coverage. 
# Particularly, if our samples are suffering from PCR bias it would be useful to 
# discard bases with very high read coverage. Furthermore, we would also like to 
# discard bases that have low read coverage, a high enough read coverage will 
#increase the power of the statistical tests. The code below filters a methylRawList 
#and discards bases that have coverage below 10X and also discards the bases that 
#have more than 99.9th percentile of coverage in each sample.

## filtra low coverage (10X) y bias de 99.9% en alguna muestra
%%R
CG.filtered=filterByCoverage(CG.cov,lo.count=10,lo.perc=NULL,
                                          hi.count=NULL,hi.perc=99.9)
CHG.filtered=filterByCoverage(CHG.cov,lo.count=10,lo.perc=NULL,
                                          hi.count=NULL,hi.perc=99.9)
CHH.filtered=filterByCoverage(CHH.cov,lo.count=10,lo.perc=NULL,
                                          hi.count=NULL,hi.perc=99.9)

## normalization step
## These two functions will help reduce the bias in the statistical tests that 
## might occur due to systematic over-sampling of reads in certain samples.
# bajará el coverage ? si lo hace, disminuye el error tipo 1
CG.norm <- unite(normalizeCoverage(CG.filtered))
CHG.norm <- unite(normalizeCoverage(CHG.filtered))
CHH.norm <- unite(normalizeCoverage(CHH.filtered))


<a name="step-2.4"></a>
## Caracterización de la metilación global

Se generarán 3 gráficos para caracterizar la metilación global:
- Histograma: distribución de C's según el % metilación en cada contexto
- Violines: distribución de C's según el % metilación en cada contexto
- Heatmaps: % de metilación para genes, elementos transponibles, repeticiones y regiones no clasificadas.

In [ ]:
#@title PCA
%%R
PCASamples(CG.norm)
PCASamples(CHG.norm)
PCASamples(CHH.norm)

pdf(paste0("/content/plots/PCA.pdf"))
PCASamples(CG.norm)
PCASamples(CHG.norm)
PCASamples(CHH.norm)
dev.off()


<a name="step-2.5"></a>
## Citosinas diferencialmente metiladas

In [ ]:
#@title Cálculo DMC
%%R
options(warn=-1)
CG.DMC <- data.frame(calculateDiffMeth(CG.norm))
CHG.DMC <- data.frame(calculateDiffMeth(CHG.norm))
CHH.DMC <- data.frame(calculateDiffMeth(CHH.norm))


In [ ]:
%%R
a<-subset(CG.DMC, CG.DMC$qvalue<0.05 & CG.DMC$meth.diff>=25) #hiper a 25
nrow(a)
a

In [ ]:
#@title Cálculo de DMR
## Hace el methylation calling en bins (o tiles) en lugar de por base para cada (se puede variar el win size)
%%R
options(warn=-1)
CG.wind <- tileMethylCounts(CG.norm, win.size = 100, step.size = 100)
CHG.wind <- tileMethylCounts(CHG.norm, win.size = 100, step.size = 100)
CHH.wind <- tileMethylCounts(CHH.norm, win.size = 100, step.size = 100)

CG.DMR <- data.frame(calculateDiffMeth(CG.wind))
CHG.DMR <- data.frame(calculateDiffMeth(CHG.wind))
CHH.DMR <- data.frame(calculateDiffMeth(CHH.wind))


In [ ]:
#@title Manhattan plot
%%R
layout(matrix(1:2, nrow = 1, byrow = TRUE))
#snpOfInterest<-intersect(DMCfile$chr,onlyDM$chr)
#snpOfInterest<-sort(snpOfInterest)
#snpOfInterest
CG.DMC$chr<-as.numeric(CG.DMC$chr)
manhattan(CG.DMC, chr = "chr", bp = "start", p = "qvalue",
          snp = "chr", col = c("gray60", "black"), 
          chrlabs = NULL, main = "DMC", 
          suggestiveline = -log10(5e-02), xlab="cromosoma", ylab=expression('-log'[10]*' (qvalue)'))
CG.DMR$chr<-as.numeric(CG.DMR$chr)
manhattan(CG.DMR, chr = "chr", bp = "start", p = "qvalue",
          snp = "chr", col = c("gray60", "black"), 
          chrlabs = NULL, main = "DMR", 
          suggestiveline = -log10(5e-02), xlab="cromosoma", ylab=expression('-log'[10]*' (qvalue)'))


In [ ]:
%%R
#@title Anotación para DMC y DMR
inFileAnnotation<-("/content/rawData/Vitis_vinifera_annotation.gff")


<a name="step-3"></a>
# Guardar en Drive

Opcional. 
Depende del espacio disponible en Drive.
Si desea copiar los datos, debe eliminar el numeral (`#`) de cada línea de código EXCEPTO la primera ``#@title``

In [ ]:
#@title Guardar archivos en el Drive personal
%%bash
#mkdir ../drive/MyDrive/EpiEvo/TP_3
#cd ..
#ls
#cp -r {plots,rawData,results} drive/MyDrive/EpiEvo/TP_3/
#ls ../drive/MyDrive/EpiEvo/TP_3